# Evolutionary Algorithm 
### How it works:
1. Initialization:

- A population of individuals is created, where each individual represents a set of hyperparameters (C and gamma in this case).
2. Evaluation:

- Each individual is evaluated using the evaluate function, which trains an SVM and computes its cross-validation accuracy.
3. Selection:

- The best individuals are selected for the next generation using tournament selection.
4. Crossover and Mutation:

- Crossover and mutation are applied to create new individuals (offspring).
5. Iteration:

- This process is repeated for a fixed number of generations.
6. Result:

- At the end, the best individual (i.e., the best set of hyperparameters) and its performance score are printed.

In [2]:
import torch

# Define the objective function (to minimize)
def objective_function(x):
    return (x - 2) ** 2  # Has a minimum at x = 2

# Function to create an initial population
def create_population(pop_size, lower_bound, upper_bound):
    return torch.rand(pop_size) * (upper_bound - lower_bound) + lower_bound

# Function to evaluate fitness (lower objective value = higher fitness)
def evaluate_fitness(population):
    return -objective_function(population)  # Use negative objective for maximizing fitness

# Function to select parents based on fitness (roulette wheel selection)
def select_parents(population, fitness, num_parents):
    probabilities = fitness - fitness.min()  # Ensure all fitness values are non-negative
    probabilities = probabilities / probabilities.sum()  # Normalize to probabilities
    selected_indices = torch.multinomial(probabilities, num_samples=num_parents, replacement=True)
    return population[selected_indices]

# Function to perform crossover between parents to create offspring
def crossover(parents, offspring_size):
    offspring = torch.empty(offspring_size)
    for i in range(offspring_size):
        # Randomly select two parents
        parent1_idx = torch.randint(0, len(parents), (1,))
        parent2_idx = torch.randint(0, len(parents), (1,))
        # Perform single-point crossover
        alpha = torch.rand(1)  # Weight for linear combination
        offspring[i] = alpha * parents[parent1_idx] + (1 - alpha) * parents[parent2_idx]
    return offspring

# Function to mutate offspring
def mutate(offspring, mutation_rate, lower_bound, upper_bound):
    for i in range(len(offspring)):
        if torch.rand(1).item() < mutation_rate:  # Apply mutation with a certain probability
            mutation = torch.randn(1).item() * 0.2  # Convert mutation to scalar with .item()
            offspring[i] += mutation  # Add the scalar mutation
            # Clip to bounds
            offspring[i] = torch.clamp(offspring[i], lower_bound, upper_bound)
    return offspring

# Main function: Evolutionary Algorithm
def evolutionary_algorithm(pop_size, num_generations, lower_bound, upper_bound, mutation_rate):
    # Step 1: Create an initial population
    population = create_population(pop_size, lower_bound, upper_bound)
    
    for generation in range(num_generations):
        # Step 2: Evaluate fitness of the population
        fitness = evaluate_fitness(population)
        
        # Step 3: Select parents
        num_parents = pop_size // 2
        parents = select_parents(population, fitness, num_parents)
        
        # Step 4: Generate offspring through crossover
        offspring_size = pop_size - num_parents  # Maintain population size
        offspring = crossover(parents, offspring_size)
        
        # Step 5: Apply mutation to offspring
        offspring = mutate(offspring, mutation_rate, lower_bound, upper_bound)
        
        # Step 6: Create the new population (parents + offspring)
        population = torch.cat((parents, offspring))
        
        # Step 7: Track the best solution
        best_fitness = fitness.max()
        best_solution = population[fitness.argmax()]
        print(f"Generation {generation + 1}: Best Solution = {best_solution.item():.4f}, Fitness = {best_fitness.item():.4f}")
    
    # Return the best solution found
    final_fitness = evaluate_fitness(population)
    best_index = final_fitness.argmax()
    best_solution = population[best_index]
    return best_solution

# Run the evolutionary algorithm
if __name__ == "__main__":
    best_solution = evolutionary_algorithm(
        pop_size=20,           # Population size
        num_generations=50,    # Number of generations
        lower_bound=0.0,       # Lower bound of the search space
        upper_bound=4.0,       # Upper bound of the search space
        mutation_rate=0.2      # Mutation rate (probability of mutation per individual)
    )
    print(f"Optimal Solution = {best_solution.item():.4f}, Objective Value = {objective_function(best_solution).item():.4f}")


Generation 1: Best Solution = 2.4228, Fitness = -0.0000
Generation 2: Best Solution = 2.0003, Fitness = -0.0000
Generation 3: Best Solution = 1.7298, Fitness = -0.0000
Generation 4: Best Solution = 1.6855, Fitness = -0.0000
Generation 5: Best Solution = 1.6855, Fitness = -0.0000
Generation 6: Best Solution = 1.6855, Fitness = -0.0000
Generation 7: Best Solution = 1.8306, Fitness = -0.0001
Generation 8: Best Solution = 1.9049, Fitness = -0.0050
Generation 9: Best Solution = 2.0368, Fitness = -0.0002
Generation 10: Best Solution = 1.8507, Fitness = -0.0000
Generation 11: Best Solution = 1.9222, Fitness = -0.0000
Generation 12: Best Solution = 2.0213, Fitness = -0.0000
Generation 13: Best Solution = 2.0268, Fitness = -0.0000
Generation 14: Best Solution = 2.0012, Fitness = -0.0000
Generation 15: Best Solution = 2.0037, Fitness = -0.0000
Generation 16: Best Solution = 2.0011, Fitness = -0.0000
Generation 17: Best Solution = 2.0011, Fitness = -0.0000
Generation 18: Best Solution = 2.0004, F

#### Advantages of Evolutionary Algorithms

- Can handle large and complex parameter spaces effectively.
- Suitable for non-continuous or discrete hyperparameters.
- Does not require a full grid or random search, saving computational effort.

#### Disadvantages

- Computationally expensive for very large populations or many generations.
- Performance depends on properly tuning the genetic algorithm's parameters (e.g., population size, mutation rate).